- https://github.com/shs1234/TIL/blob/master/project/Process.ipynb

In [1]:
import requests
import json
import sys
import cv2
import matplotlib.pyplot as plt
import pytesseract
import numpy as np
import pandas as pd
import datetime
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'


In [2]:
def time_msec(time):
    time_split = time.split(':')
    if len(time_split)==2:
        sec = int(time_split[0])*60 + int(time_split[1])
        return sec * 1000
    elif len(time_split) == 3:
        sec = (int(time_split[0]) * 60 + int(time_split[1]) * 60 + int(time_split[2]))
        return sec * 1000
    
    
def msec_time(msec):
    sec = msec/1000
    return str(int(sec//60) + ':' + str(int(sec%60)))

# 1. 트위치 채팅 내역 수집

In [3]:
def doubleDigit(num):
    if num < 10:
        return '0' + str(num)
    else:
        return str(num)
    

In [4]:
def twichchat(videoId,clientId):
#     videoId = '571178493'
#     clientId = 'vhggq5l1jefc8dtfza2phfc6ih69gs'
    
    videoId = videoId
    clientId = clientId
    
    chat = []
    time = []
    user = []

    nextCursor = ''

    params = {}
    params['client_id'] = clientId

    i = 0
    while True :

        if i == 0 :
            URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?content_offset_seconds=0' 
            i += 1
        else:
            URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?cursor=' 
            URL +=  nextCursor   


        response = requests.get(URL, params=params)

        j = json.loads(response.text)

        for k in range(0,len(j["comments"])):
            timer = j["comments"][k]["content_offset_seconds"]

            timeMinute = int(timer/60)

            if timeMinute >= 60 :
                timeHour = int(timeMinute/60)
                timeMinute %= 60
            else :
                timeHour = int(timeMinute/60)

            timeSec = int(timer%60)

            time.append(doubleDigit(timeHour)+':'+doubleDigit(timeMinute)+':'+doubleDigit(timeSec))
            user.append(j["comments"][k]["commenter"]["display_name"])
            chat.append(j["comments"][k]["message"]["body"])
        if '_next' not in j:
            break

        nextCursor = j["_next"]
    return time

    f = open(videoId+".txt", 'wt', encoding="utf-8")

    for x in range(0, len(time)): 
            f.write('[')
            f.write(str(time[x]))
            f.write(']')
            f.write(' ')
            f.write('<')
            f.write(str(user[x]))
            f.write('>')
            f.write(' ')
            f.write(str(chat[x]))
            f.write("\n")

    f.close()

In [5]:
videoId = '571178493'
clientId = 'vhggq5l1jefc8dtfza2phfc6ih69gs'


In [6]:
timedata = twichchat(videoId, clientId)

KeyError: 'comments'

In [7]:
time = timedata[:]


In [8]:
time

['00:00:04',
 '00:00:05',
 '00:00:07',
 '00:00:07',
 '00:00:08',
 '00:00:09',
 '00:00:12',
 '00:00:19',
 '00:00:19',
 '00:00:21',
 '00:00:34',
 '00:00:34',
 '00:00:39',
 '00:00:41',
 '00:00:42',
 '00:00:45',
 '00:00:47',
 '00:00:47',
 '00:00:48',
 '00:00:49',
 '00:00:51',
 '00:00:52',
 '00:00:55',
 '00:00:56',
 '00:00:58',
 '00:01:01',
 '00:01:02',
 '00:01:02',
 '00:01:03',
 '00:01:06',
 '00:01:06',
 '00:01:07',
 '00:01:08',
 '00:01:08',
 '00:01:09',
 '00:01:11',
 '00:01:12',
 '00:01:12',
 '00:01:13',
 '00:01:14',
 '00:01:14',
 '00:01:14',
 '00:01:15',
 '00:01:15',
 '00:01:16',
 '00:01:16',
 '00:01:16',
 '00:01:18',
 '00:01:19',
 '00:01:19',
 '00:01:20',
 '00:01:20',
 '00:01:20',
 '00:01:21',
 '00:01:21',
 '00:01:22',
 '00:01:22',
 '00:01:22',
 '00:01:22',
 '00:01:23',
 '00:01:24',
 '00:01:26',
 '00:01:26',
 '00:01:26',
 '00:01:26',
 '00:01:28',
 '00:01:28',
 '00:01:28',
 '00:01:29',
 '00:01:29',
 '00:01:30',
 '00:01:30',
 '00:01:30',
 '00:01:31',
 '00:01:32',
 '00:01:33',
 '00:01:33',

In [9]:
for i in range(len(time)):
    time[i] = time[i][1:]

# 2. OCR을 이용해 영상에서 정보 추출

In [10]:
def ocr1(frame):
#     스코어 ocr용 함수
    board=frame[:25,1550:1905]
    width = board.shape[1]
    score=board[:,5:int(width*0.21)]

    sharpening = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    sharp = cv2.filter2D(score, -1, sharpening)

    fx=4
    fy=4
    resize=cv2.resize(sharp, dsize=(0,0), fx=fx,fy=fy)

    gray = cv2.cvtColor(resize, cv2.COLOR_RGB2GRAY)

    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    _, th = cv2.threshold(blur, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    
    str_score = [pytesseract.image_to_string(th)]
    return str_score

In [11]:
def ocr2(frame):
#     kda, 시간 ocr 함수
    board=frame[:25,1550:1905]
    gray = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)

    fx=4
    fy=4

    board=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)

    img_blur = cv2.GaussianBlur(board, (3,3), 0)
    _, board = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    width = board.shape[1]

    kda = board[:,int(width*0.32):int(width*0.48)]
    
    data = []
    str_kda = pytesseract.image_to_string(kda)
    data.append(str_kda)
    
    time = board[:,int(width*0.87):]
    str_time = pytesseract.image_to_string(time)
    data.append(str_time)

    return data

In [12]:
def ocr3(frame):
#     골드 ocr 함수
    gold=frame[1047:1070,1180:1237]
    gray = cv2.cvtColor(gold, cv2.COLOR_RGB2GRAY)
    
    fx=4
    fy=4
    resize=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)
    blur = cv2.GaussianBlur(resize, (3, 3), 0)
    _, th = cv2.threshold(blur, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)

    str_gold = [pytesseract.image_to_string(th)]
    return str_gold

In [14]:
# 동영상 프로세싱
cap = cv2.VideoCapture('video/full.mp4')

starttime = '2:06:38'
endtime = '2:22:11'

timemsec = time_msec(starttime)
cap.set(cv2.CAP_PROP_POS_MSEC, timemsec)

i=0
datalist=[]
while (cap.isOpened):
    i += 1
    ret, frame = cap.read()
    if ret:
        score = ocr1(frame)
        kda_time = ocr2(frame)
        gold = ocr3(frame)
        vid_time = [cap.get(cv2.CAP_PROP_POS_MSEC)]
        data = score + kda_time + gold + vid_time
        datalist.append(data)
        if i%60 == 0:
            print(data)
        
        timemsec += 1000
        cap.set(cv2.CAP_PROP_POS_MSEC, timemsec)
        
        if vid_time[0] > time_msec(endtime):
            break
    else:
        break